# Test model 

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

import string
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem.snowball import SnowballStemmer 
from pymystem3 import Mystem
mystem = Mystem() 


russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
stemmer = SnowballStemmer("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
# Text clean function
def clean_text(text, remove_sw=True, remove_stem=True, remove_lemm=False):
    # Text to lower index
    text = text.lower()

    # Remove punkruation and numbers from the text
    remove_punkt_digets = ''
    for char in text:
        if char not in string.punctuation and not char.isdigit():
            remove_punkt_digets += char
        else:
            remove_punkt_digets += ' '

    # Remove multiple spaces from the text
    text = re.sub(r'\s+', ' ', remove_punkt_digets, flags=re.I)

    # Remove stopwords from the text (optional)
    if remove_sw:
        clean_text = ''
        tokens = word_tokenize(text)  # tokenize text string
        for token in tokens:
            if token not in russian_stopwords and token != ' ':
                clean_text = clean_text + ' ' + token
        text = clean_text.strip()

    # Stemming   
    if remove_stem:
      stemmed_tokens = []
      tokens = word_tokenize(text) # tokenize text string
      for token in tokens:
        stemmed_tokens.append(stemmer.stem(token)) 
      text = " ".join(stemmed_tokens)
      text = text.strip()

    # Lemmtization 
    if remove_lemm:
      lemmed_tokens = []
      tokens = word_tokenize(text) # tokenize text string
      for token in tokens:
        lemmed_tokens.append(mystem.lemmatize(token)[0])
      text = " ".join(lemmed_tokens)
      text = text.strip()

    return text

In [ ]:
# Load the pretrained tokenizer
with open('/content/drive/MyDrive/tokenizer/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
#tokenizer.word_index

In [ ]:
# Dictionary of labels to predict
labels_dict = {0: 'Бывший СССР', 
              1: 'Из жизни', 
              2: 'Интернет и СМИ', 
              3: 'Культура', 
              4: 'Мир', 
              5: 'Наука и техника', 
              6: 'Россия', 
              7: 'Силовые структуры', 
              8: 'Спорт', 
              9: 'Экономика'}

### Load the train model and testing on random article

In [ ]:
# Import pretarined CNN model  
file_path = "/content/drive/MyDrive/best_model/model_best_val.h5"  
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model(file_path)

In [ ]:
# Check model structure
reconstructed_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 200)          50000000  
                                                                 
 conv1d_8 (Conv1D)           (None, 498, 16)           9616      
                                                                 
 batch_normalization_8 (Batc  (None, 498, 16)          64        
 hNormalization)                                                 
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 249, 16)          0         
 1D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 249, 16)           0         
                                                                 
 conv1d_9 (Conv1D)           (None, 247, 32)          

In [ ]:
# Text of the article
news_article = 'Директор Государственного музея изобразительных искусств \
имени Пушкина Ирина Антонова названа лауреатом за 2000 год премии имени \
Шостаковича, учрежденной  Международным благотворительным фондом Юрия \
Башмета в 1994 году. Премия  вручается раз в год одному человеку за \
выдающиеся достижения в области искусства на  протяжении всей жизни. \
До сих пор ее лауреатами становились исключительно музыканты: скрипачи \
Гидон Кремер и Анна-Софи Муттер, виолончелистка Наталия Гутман, \
вокалисты  Томас Квастхофф (бас-баритон) и Ольга Бородина (меццо-сопрано), \
дирижер, художественный руководитель Мариинского театра  Валерий \
Гергиев. Церемония вручения премии имени Шостаковича Ирине Антоновой \
состоится 14 мая. Денежное наполнение премии составляет 25 тысяч долларов. \
Это самая большая из российских "культурных" премий.'

In [ ]:
news_article2 = 'На берлинской международной авиационно-космической выставке "ИЛА-2006" Россия продемонстрирует новейший истребитель МиГ-29М (МиГ-35), сообщает пресс-служба ФСВТС России. МиГ-29М, разработка которого началась в середине 80-х годов XX века, является последней и наиболее совершенной модификацией платформы МиГ-29. Истребитель имеет цифровую электродистанционную систему управления, оснащен двигателями с изменяемым вектором тяги, новейшей системой управления огнем, включающей в себя усовершенствованную оптико-локационную станцию, позволяющую обнаруживать воздушные цели на значительном расстоянии не выдавая себя работой РЛС, и новейший радар, позволяющий обнаруживать различные типы воздушных, наземных и морских целей, в том числе построенных с применением технологий "Стелс". Истребитель будет способен применять все новейшие средства авиационного поражения, разработанные для машин фронтовой авиации, в том числе ракеты воздух-воздух РВВ-АЕ и воздух-земля Х-59М "Овод". По сравнению с МиГ-29 серий 9-12 и 9-13, состоящих сейчас на вооружении ВВС России, МиГ-29М имеет повышенную дальность полета - за счет более экономичных двигателей и наличия системы дозаправки в воздухе, большую бомбовую нагрузку за счет увеличения числа узлов подвески и более высокую маневренность. Также новый "МиГ" оснащен "интеллектуальной кабиной" с жидкокристаллическими панелями на приборной доске и нашлемной системой целеуказания. В конструкции планера истребителя применены радиопоглощающие материалы, снижающие его заметность для радаров противника. Выставка "ILA-2006" является одним из крупнейших авиационных форумов в мире. В нынешнем году она будет проводиться в восьмой раз.'

In [ ]:
#                 Predict
def label_predict(text):
  # Clean text
  clean_article = clean_text(text)

  # Transform text to pad sequence
  seq = tokenizer.texts_to_sequences([clean_article])
  padded = pad_sequences(seq, maxlen=500, value=0.0,
                         padding='post', truncating='post')

  # Eval predict label
  pred = reconstructed_model.predict(padded)

  # Label name
  predicted_label = labels_dict[np.argmax(pred[0])]

  print(f'Predicted label is: {predicted_label}')

In [ ]:
label_predict(news_article)

Predicted label is: Культура


In [ ]:
label_predict(news_article2)

Product category id: 5
Predicted label is: Наука и техника


In [ ]:
df_res = pd.read_csv('/content/drive/MyDrive/lenta_parser/lenta_parser.csv')

In [ ]:
df_res

,Title,Description,Links,Publication Date
0,Самый удаленный от Земли зонд начал присылать ...,Национальное управление по аэронавтике и иссле...,https://lenta.ru/news/2022/05/21/nasa_/,"Sat, 21 May 2022 01:39:21 +0300"
1,Германия и Италия разрешили открывать счета в ...,Власти Германии и Италии разрешили своим компа...,https://lenta.ru/news/2022/05/21/gas/,"Sat, 21 May 2022 01:38:26 +0300"
2,Названа причина остановки производства вакцин ...,Замдиректора по научной работе ЦНИИ эпидемиоло...,https://lenta.ru/news/2022/05/21/proizvodstvo_...,"Sat, 21 May 2022 01:35:08 +0300"
3,Назван способ определить ложный звонок «из пол...,Заслуженный юрист России Иван Соловьев рассказ...,https://lenta.ru/news/2022/05/21/call/,"Sat, 21 May 2022 01:33:00 +0300"
4,В России напомнили о провокации продовольствен...,Первый заместитель постоянного представителя Р...,https://lenta.ru/news/2022/05/21/zapad/,"Sat, 21 May 2022 01:23:49 +0300"
...,...,...,...,...
192,Создан эффективный способ лечения смертоносной...,Ученые Женской больницы Бригама в Бостоне созд...,https://lenta.ru/news/2022/05/19/celltherapy/,"Fri, 20 May 2022 16:30:00 +0300"
193,Семья американцев умерла при загадочных обстоя...,Американская семья отправилась в отпуск на рос...,https://lenta.ru/news/2022/05/20/familydeath/,"Fri, 20 May 2022 16:30:00 +0300"
194,Повара посоветовали необычный способ усилить в...,Повара посоветовали натирать посуду чесноком д...,https://lenta.ru/news/2022/05/20/chesnok/,"Fri, 20 May 2022 16:19:00 +0300"
195,Путин поручил оценить антироссийские меры ВТО,Российский лидер Владимир Путин поручил правит...,https://lenta.ru/news/2022/05/20/putin_analiz/,"Fri, 20 May 2022 16:15:24 +0300"


In [ ]:
classified_news = pd.DataFrame(columns=['text', 'topic'])
predicted_articles = []
predicted_labels = []
for article in df_res['Description']:
  # Clean text
  clean_article = clean_text(article)

  # Transform text to pad sequence
  seq = tokenizer.texts_to_sequences([clean_article])
  padded = pad_sequences(seq, maxlen=500, padding='post', truncating='post')  

  # Eval predict label
  pred = reconstructed_model.predict(padded)

  # Label name
  predicted_label = labels_dict[np.argmax(pred[0])]

  predicted_articles.append(article)
  predicted_labels.append(predicted_label)

classified_news['text'] = predicted_articles
classified_news['topic'] = predicted_labels

In [ ]:
classified_news

,text,topic
0,Национальное управление по аэронавтике и иссле...,Наука и техника
1,Власти Германии и Италии разрешили своим компа...,Россия
2,Замдиректора по научной работе ЦНИИ эпидемиоло...,Наука и техника
3,Заслуженный юрист России Иван Соловьев рассказ...,Наука и техника
4,Первый заместитель постоянного представителя Р...,Наука и техника
...,...,...
192,Ученые Женской больницы Бригама в Бостоне созд...,Наука и техника
193,Американская семья отправилась в отпуск на рос...,Наука и техника
194,Повара посоветовали натирать посуду чесноком д...,Культура
195,Российский лидер Владимир Путин поручил правит...,Экономика


In [ ]:
classified_news['topic'].unique()

array(['Наука и техника', 'Россия', 'Силовые структуры', 'Из жизни',
       'Бывший СССР', 'Экономика', 'Мир', 'Спорт', 'Культура',
       'Интернет и СМИ'], dtype=object)

# Test tflite model

In [ ]:
# Covert to .tflite
converter = tf.lite.TFLiteConverter.from_saved_model(file_path2)
tflite_model = converter.convert()

In [ ]:
import pathlib

# Save tflite model
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

76928292

In [ ]:
# Load saved model to interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

In [ ]:
# Look at details for understand format of input data
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print()
print(output_details)

[{'name': 'serving_default_embedding_5_input:0', 'index': 0, 'shape': array([  1, 200], dtype=int32), 'shape_signature': array([ -1, 200], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

[{'name': 'StatefulPartitionedCall:0', 'index': 18, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
# We look at input_details 'shape': array([  1, 200] and 'dtype': <class 'numpy.float32'>
# Use padded (it's our transform text with shape(1, 200)), exactly what is needed
to_predict = np.array([[padded]], dtype=np.float32)
print(to_predict)

[[[[1.36000e+04 5.20340e+04 1.13600e+03 1.07000e+02 5.35200e+03
    1.06400e+03 1.86640e+04 4.97900e+04 1.47000e+02 7.28000e+02
    5.47900e+03 2.09410e+04 1.00025e+05 2.57140e+04 6.50300e+03
    5.72000e+02 6.18000e+02 1.27100e+03 3.66100e+03 7.40700e+03
    5.40000e+02 1.55100e+03 3.40000e+01 7.80000e+01 5.80000e+01
    1.02200e+03 5.32400e+03 8.99800e+03 1.10640e+04 8.93710e+04
    6.90000e+01 7.22000e+02 2.29000e+02 1.90000e+02 1.02200e+03
    2.55750e+04 5.83170e+04 6.20000e+01 1.00000e+00 1.70320e+04
    2.21310e+04 9.93000e+02 8.53600e+04 1.06400e+03 3.29360e+04
    5.02000e+03 3.50200e+03 8.93710e+04 2.21000e+03 1.90820e+04
    5.40000e+02 1.00030e+04 2.19430e+04 7.86000e+02 4.33800e+03
    1.19330e+04 1.46600e+03 9.42600e+03 6.39500e+03 4.51000e+02
    6.18000e+02 5.17000e+02 4.40000e+02 1.23300e+03 1.27900e+03
    6.00000e+01 8.59000e+02 8.71000e+02 1.64000e+03 1.95840e+04
    1.63750e+04 9.95800e+03 1.49200e+03 6.27800e+03 2.96000e+02
    5.09000e+02 1.46000e+02 5.30200e+03 

In [ ]:
# to_predict have shape(1, 1, 1, 200), but we need a shape(1, 200), (look input_details 'shape')
# So, we just take to_predict[0][0]
to_predict.shape

(1, 1, 1, 200)

In [ ]:
# Set an input tensor and call the interpreter
interpreter.set_tensor(input_details[0]['index'], to_predict[0][0])
interpreter.invoke()

In [ ]:
# Get a result tensor
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[2.1825507e-04 7.5461329e-03 2.4342681e-03 9.8625529e-01 1.5844185e-03
  2.5005580e-04 1.5650622e-03 1.0665232e-04 3.2020907e-05 7.8833200e-06]]


In [ ]:
# The model return shape(1, 10) vector
tflite_results.shape

(1, 10)

In [ ]:
# To predict the label take a max value of vector
predict_label = np.argmax(tflite_results[0])

In [ ]:
print(f'Product category id: {predict_label}')
print(f'Predicted label is: {labels_dict[predict_label]}')

Product category id: 3
Predicted label is: Культура
